In [ ]:
!pip install langchain -q
!pip install pdfminer.six -q
!pip install unstructured -q
!pip install pinecone-client -q
!pip install pdf2image -q
!pip install pytesseract -q
!pip install tiktoken -q
!pip install langchain-community
!pip install --quiet langchain-google-genai
!pip install langchain-pinecone

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 487.7 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 997.8/997.8 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 391.5/391.5 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.4/140.4 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 6.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.4/431.4 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.7/274.7 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 33.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 2.0 MB/s eta 0:00:00
   

In [ ]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import TextLoader
from langchain.vectorstores import Pinecone

In [ ]:
from pinecone import Pinecone
import os
from google.colab import userdata
pc = Pinecone(api_key=userdata.get('pinecone_key'))
index = pc.Index("data")

In [ ]:
from langchain import PromptTemplate
from langchain.schema import StrOutputParser
from langchain.schema.prompt_template import format_document
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_google_genai import GoogleGenerativeAIEmbeddings

In [ ]:
os.environ['GOOGLE_API_KEY'] = userdata.get('gemini_key')

In [ ]:
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash",temperature=0.8)

In [ ]:
gemini_embeddings = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004")

In [ ]:
from langchain_pinecone import PineconeVectorStore
vector_store = PineconeVectorStore(index=index, embedding=gemini_embeddings)

# Demo

**Test Queries:**

How much was the revenue for TCS in Q3 2023?

Products and platforms that saw good traction in TCS Q3 2023?

Revenue growth for TCS in Q4 2023?

Generative AI advancements made by TCS in Q4 2023-2024?

Generative AI advancements made by TCS in Q1 2024-2025?



docid-1 = Transcript of the Q1 2024-25 Earnings Conference Call held on Jul 11, 2024

docid-2 = Transcript of the Q4 2023-24 Earnings Conference Call held on April 12, 2024

docid-3 = Transcript of the Q3 2023-24 Earnings Conference Call held on January 11, 2023

In [46]:
user_query = "Generative AI advancements made by TCS in Q1 2024-2025?"

llm_completion_select_route_chain = (
PromptTemplate.from_template("""
Given the user question below, classify whether its a general query requiring text generation tasks or a document retrieval query requiring a text extraction task for answering the user question.

General: If the question involves generating new text, such as creating, summarizing, or explaining content that isn't directly tied to specific documents.

Document: If the question requires extracting specific information from an existing document or dataset, indicating that the answer must be pulled directly from a particular source.

<question>
{question}
</question>

The response must contain only the classified category - General or Document.

<question>
{question}
</question>

Classification:""")
| llm
| StrOutputParser()
)
response=llm_completion_select_route_chain.invoke({"question": user_query})
if response.strip()=="Document":
  matched_summaries=vector_store.similarity_search(
      user_query,  # our search query
      k=3 , # return 3 most relevant docs
      namespace="summary_embeddings"
  )
  summaries=[]
  for passage in matched_summaries:
    summaries.append(passage.metadata["source"]+": "+passage.page_content)
  summary_list=""
  for i in range(0,len(summaries)):
      summary_list=summary_list+summaries[i]+"###########\n\n"
  document_selection_prompt = f"""You are an AI assistant that can identify which documents have the probable answer to the user questions. You will be given passages each containing the summary of the of the document. Based on the user question you have to reply with a python list containing the document ids. The document ids will be like docid-1,docid-2,docid-3,docid-4 and docid-5. Only single document can have the answer or multiple documents can also have the answer.
  Return only the document ids strictly and nothing else.
  Examples:
  ["docid-1"]
  ["docid-1","docid-2"]
  ["docid-1","docid-2","docid-3"]
  ["docid-2"]
  []

  Document Summaries:

  {summary_list}
  """
  messages = [
      (
          "system",
        document_selection_prompt,
      ),
      ("human", user_query),
  ]
  response = llm.invoke(messages)
  import json
  selected_doc=json.loads(response.content.strip())
  print(selected_doc)
  matched_passages=vector_store.similarity_search(
      user_query,  # our search query
      k=5 , # return 5 most relevant docs
      namespace="passages_embeddings",
      filter={"source":{"$in": selected_doc}}
  )
  paragraphs_list=""
  for i in range(0,5):
      paragraphs_list+=matched_passages[i].page_content+"\n\n"
  final_answer_generation_prompt="""You are an AI assistant that can provide helpful information to the user. You are given the following extracted parts of a long document and a question. Answer the question with the help of the supporting texts. Also mention the document to the user from which the answer was generated so that he can refer to it."""
  user_prompt = f"""
  Question: {user_query}
  ======
  Supporting texts:
  {paragraphs_list}

  Reffered Document:
  {str(selected_doc)}
  ======

  Answer:
  """
  messages = [
      (
          "system",
        final_answer_generation_prompt,
      ),
      ("human", user_prompt),
  ]
  response = llm.invoke(messages)
  print(response.content)
else:
  messages = [
      (
          "system",
        "You are an AI assistant that can respond to any user query.",
      ),
      ("human", user_query),
  ]
  response = llm.invoke(messages)
  print(response.content)

['docid-1']
The document provided focuses on TCS's earnings conference call for the first quarter of 2024-2025. It discusses the company's overall performance and the impact of Generative AI on its business.  However, it does not mention any specific advancements in GenAI made by TCS during this period. 

Therefore, based on the provided document, I cannot answer your question about specific Generative AI advancements made by TCS in Q1 2024-2025. 

